# Life-Cycle Labor Supply of Couples with Children

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

# Dual-Earner Model with Children

In [ ]:
# load local model file and initialize model class
from DynHouseholdLaborModel import DynHouseholdLaborModelClass
model = DynHouseholdLaborModelClass()

par = model.par
sol = model.sol
sim = model.sim

In [ ]:
%time model.solve()

In [ ]:
k1_mesh,k2_mesh = np.meshgrid(par.k_grid,par.k_grid,indexing='ij')

# first period
t = 0
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(k1_mesh, k2_mesh, sol.h1[t,0],rstride=1,cstride=1,cmap='viridis', edgecolor='none',alpha=0.5);
ax.plot_surface(k1_mesh, k2_mesh, sol.h1[t,1],rstride=1,cstride=1,cmap='gray', edgecolor='none',alpha=0.5);
ax.set(xlabel='Wealth, $k_{1,T}$',ylabel='Human capital, $k_{2,T}$',zlabel='Hours, $h_{T}^{\star}$');

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(k1_mesh, k2_mesh, sol.V[t,0],rstride=1,cstride=1,cmap='viridis', edgecolor='none',alpha=0.5);
ax.plot_surface(k1_mesh, k2_mesh, sol.V[t,1],rstride=1,cstride=1,cmap='gray', edgecolor='none',alpha=0.5);
ax.set(xlabel='Wealth, $a_{T}$',ylabel='Human capital, $k_T$',zlabel='Value, $V_{T}$');

# Simulate child-related transfers reforms

In [ ]:
model.simulate()

In [ ]:
# plot behavior
for var in ('h1','h2','k1','k2','income1','income2'):
    fig, ax = plt.subplots()
    ax.scatter(range(par.simT),np.mean(getattr(sim,var),axis=0),label='Simulated')
    ax.set(xlabel='period, t',ylabel=f'Avg. {var}',xticks=range(par.simT));

In [ ]:
#model_no_transfers = DynHouseholdLaborModelClass(par={'uncon_uni': 0.0, 'means_level': 0.0, 'means_slope': 0.0, 'cond':0.0, 'cond_high': 0.0})
model_uncon_uni = DynHouseholdLaborModelClass(par={'uncon_uni': 5000.0, 'means_level': 0.0, 'means_slope': 0.0, 'cond':0.0, 'cond_high': 0.0})
model_uncon_means = DynHouseholdLaborModelClass(par={'uncon_uni': 0.0, 'means_level': 10000.0, 'means_slope': 5.0, 'cond':0.0, 'cond_high': 0.0})
model_con_uni = DynHouseholdLaborModelClass(par={'uncon_uni': 0.0, 'means_level': 0.0, 'means_slope': 0.0, 'cond':8000.0, 'cond_high': -400.0})
model_con_means = DynHouseholdLaborModelClass(par={'uncon_uni': 0.0, 'means_level': 0.0, 'means_slope': 0.0, 'cond':8000.0, 'cond_high': -1500})

In [ ]:

model_uncon_uni.solve()
model_uncon_uni.simulate()

#model_uncon_means.solve()
#model_uncon_means.simulate()

model_con_uni.solve()
model_con_uni.simulate()

#model_con_means.solve()
#model_con_means.simulate()


In [ ]:
# Define the dictionary for y-axis labels
ylabels = {
    'h1': 'Average Hours Worked, Female',
    'h2': 'Average Hours Worked, Male',
    'k1': 'Average Human Capital, Female',
    'k2': 'Average Human Capital, Male',
    'income1': 'Average Labour Earnings, Female',
    'income2': 'Average Labour Earnings, Male',
    'n': 'Average Number of Children'
}

# Plot behavior
for var in ('h1', 'h2', 'k1', 'k2', 'income1', 'income2', 'n'):
    fig, ax = plt.subplots()
    # ax.plot(range(par.simT), np.mean(getattr(model_no_transfers.sim, var), axis=0), label='no transfers', marker='o')
    ax.plot(range(par.simT), np.mean(getattr(model_uncon_uni.sim, var), axis=0), label='Unconditional on market work', marker='o')
    #ax.plot(range(par.simT), np.mean(getattr(model_uncon_means.sim, var), axis=0), label='Unconditional, Means-Tested', marker='o')
    ax.plot(range(par.simT), np.mean(getattr(model_con_uni.sim, var), axis=0), label='Conditional on market work', marker='x')
    #ax.plot(range(par.simT), np.mean(getattr(model_con_means.sim, var), axis=0), label='Conditional, Means-Tested', marker='o')
    ax.set(xlabel='Period, t', ylabel=ylabels[var], xticks=range(par.simT))
    ax.legend()
    plt.show()

# Elasticity Stuff

Marshall Elasticity

Unconditional Universal

In [ ]:

# simulate Marshall elasticity in the first period
model_uncon_uni_10pct = model_uncon_uni.copy()
model_uncon_uni_10pct.par.uncon_uni = 1.10*model_uncon_uni_10pct.par.uncon_uni
model_uncon_uni_10pct.solve()
model_uncon_uni_10pct.simulate()

In [ ]:
elas_marshall_female = (model_uncon_uni_10pct.sim.h1 - model_uncon_uni.sim.h1)/model_uncon_uni.sim.h1*100
print(f'Marshall elasticity of hours worked Female: {np.mean(elas_marshall_female)}')
elas_marshall_male = (model_uncon_uni_10pct.sim.h2 - model_uncon_uni.sim.h2)/model_uncon_uni.sim.h2*100
print(f'Marshall elasticity of hours worked Male: {np.mean(elas_marshall_male)}')

In [ ]:
# Calculate the Marshall elasticity for the entire period
elas_marshall_female = (model_uncon_uni_10pct.sim.h1 - model_uncon_uni.sim.h1) / model_uncon_uni.sim.h1 * 100
elas_marshall_male = (model_uncon_uni_10pct.sim.h2 - model_uncon_uni.sim.h2) / model_uncon_uni.sim.h2 * 100

# Access the elasticity for period 5 (assuming periods are zero-indexed)
mean_period_5_female = np.mean(elas_marshall_female[:, 5])
mean_period_5_male = np.mean(elas_marshall_male[:, 5])

print(f'Marshall elasticity of hours worked Female in period 5: {mean_period_5_female}')
print(f'Marshall elasticity of hours worked Male in period 5: {mean_period_5_male}')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate the Marshall elasticity for the entire period for both females and males
elas_marshall_female = (model_uncon_uni_10pct.sim.h1 - model_uncon_uni.sim.h1) / model_uncon_uni.sim.h1 * 100
elas_marshall_male = (model_uncon_uni_10pct.sim.h2 - model_uncon_uni.sim.h2) / model_uncon_uni.sim.h2 * 100

# Calculate the mean elasticity for each period across all individuals
mean_elas_marshall_female = np.mean(elas_marshall_female, axis=0)
mean_elas_marshall_male = np.mean(elas_marshall_male, axis=0)

# Plot the average elasticity across all 10 periods
periods = np.arange(1, 11)  # Assuming there are 10 periods, and periods are 1-indexed

plt.figure(figsize=(10, 6))
plt.plot(periods, mean_elas_marshall_female, label='Female', marker='o')
plt.plot(periods, mean_elas_marshall_male, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
plt.title('Average Marshall Elasticity of Hours Worked Across Periods')
plt.legend()
plt.grid(True)
plt.xticks(periods)  # Ensure x-axis shows all periods
plt.show()


In [ ]:
# Access the first period (first column) and compute the mean
mean_first_period_female = np.mean(elas_marshall_female[:, 0])
mean_first_period_male = np.mean(elas_marshall_male[:, 0])

print(f'Frisch elasticity of hours worked Female in the first period: {mean_first_period_female}')
print(f'Frisch elasticity of hours worked Male in the first period: {mean_first_period_male}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the models and increase relevant parameters by 10%
models = {
    "uncon_uni": model_uncon_uni,
    #"uncon_means": model_uncon_means,
    #"con_uni": model_con_uni,
    "con_means": model_con_means,
}

def increase_params_and_simulate(model, params):
    model_10pct = model.copy()
    for param in params:
        setattr(model_10pct.par, param, 1.10 * getattr(model.par, param))
    model_10pct.solve()
    model_10pct.simulate()
    return model_10pct

# Models with increased parameters
model_uncon_uni_10pct = increase_params_and_simulate(model_uncon_uni, ['uncon_uni'])
model_uncon_means_10pct = increase_params_and_simulate(model_uncon_means, ['means_level', 'means_slope'])
model_con_uni_10pct = increase_params_and_simulate(model_con_uni, ['cond'])
model_con_means_10pct = increase_params_and_simulate(model_con_means, ['cond', 'cond_high'])

# Calculate elasticity for each model across all 10 periods
def calculate_elasticity(model_10pct, model):
    elas_marshall_female = (model_10pct.sim.h1 - model.sim.h1) / model.sim.h1 * 100
    elas_marshall_male = (model_10pct.sim.h2 - model.sim.h2) / model.sim.h2 * 100
    mean_elas_marshall_female = np.mean(elas_marshall_female, axis=0)
    mean_elas_marshall_male = np.mean(elas_marshall_male, axis=0)
    return mean_elas_marshall_female, mean_elas_marshall_male

mean_elas_female_uncon_uni, mean_elas_male_uncon_uni = calculate_elasticity(model_uncon_uni_10pct, model_uncon_uni)
mean_elas_female_uncon_means, mean_elas_male_uncon_means = calculate_elasticity(model_uncon_means_10pct, model_uncon_means)
mean_elas_female_con_uni, mean_elas_male_con_uni = calculate_elasticity(model_con_uni_10pct, model_con_uni)
mean_elas_female_con_means, mean_elas_male_con_means = calculate_elasticity(model_con_means_10pct, model_con_means)

# Plot the average elasticity across all 10 periods for each model
periods = np.arange(1, 11)

plt.figure(figsize=(14, 8))

plt.subplot(2, 2, 1)
plt.plot(periods, mean_elas_female_uncon_uni, label='Female', marker='o')
plt.plot(periods, mean_elas_male_uncon_uni, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
plt.title('uncon_uni')
plt.legend()
plt.grid(True)
plt.xticks(periods)

plt.subplot(2, 2, 2)
plt.plot(periods, mean_elas_female_uncon_means, label='Female', marker='o')
plt.plot(periods, mean_elas_male_uncon_means, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
plt.title('uncon_means')
plt.legend()
plt.grid(True)
plt.xticks(periods)

plt.subplot(2, 2, 3)
plt.plot(periods, mean_elas_female_con_uni, label='Female', marker='o')
plt.plot(periods, mean_elas_male_con_uni, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
plt.title('con_uni')
plt.legend()
plt.grid(True)
plt.xticks(periods)

plt.subplot(2, 2, 4)
plt.plot(periods, mean_elas_female_con_means, label='Female', marker='o')
plt.plot(periods, mean_elas_male_con_means, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
plt.title('con_means')
plt.legend()
plt.grid(True)
plt.xticks(periods)

plt.tight_layout()
plt.show()


In [ ]:
# Increase relevant parameters by 10% for each model
model_uncon_uni_10pct = model_uncon_uni.copy()
model_uncon_uni_10pct.par.uncon_uni = 1.10 * model_uncon_uni.par.uncon_uni
model_uncon_uni_10pct.solve()
model_uncon_uni_10pct.simulate()

model_uncon_means_10pct = model_uncon_means.copy()
model_uncon_means_10pct.par.means_level = 1.10 * model_uncon_means.par.means_level
model_uncon_means_10pct.par.means_slope = 1.10 * model_uncon_means.par.means_slope
model_uncon_means_10pct.solve()
model_uncon_means_10pct.simulate()

model_con_uni_10pct = model_con_uni.copy()
model_con_uni_10pct.par.cond = 1.10 * model_con_uni.par.cond
model_con_uni_10pct.solve()
model_con_uni_10pct.simulate()

model_con_means_10pct = model_con_means.copy()
model_con_means_10pct.par.cond = 1.10 * model_con_means.par.cond
model_con_means_10pct.par.cond_high = 1.10 * model_con_means.par.cond_high
model_con_means_10pct.solve()
model_con_means_10pct.simulate()

# Calculate elasticity for each model
# Marshall elasticity for model_uncon_uni
elas_marshall_female_uncon_uni = (model_uncon_uni_10pct.sim.h1 - model_uncon_uni.sim.h1) / model_uncon_uni.sim.h1 * 100
mean_first_period_female_uncon_uni = np.mean(elas_marshall_female_uncon_uni[:, 0])
elas_marshall_male_uncon_uni = (model_uncon_uni_10pct.sim.h2 - model_uncon_uni.sim.h2) / model_uncon_uni.sim.h2 * 100
mean_first_period_male_uncon_uni = np.mean(elas_marshall_male_uncon_uni[:, 0])

print(f"Marshall elasticity of hours worked Female in the first period (uncon_uni): {mean_first_period_female_uncon_uni}")
print(f"Marshall elasticity of hours worked Male in the first period (uncon_uni): {mean_first_period_male_uncon_uni}")

# Marshall elasticity for model_uncon_means
elas_marshall_female_uncon_means = (model_uncon_means_10pct.sim.h1 - model_uncon_means.sim.h1) / model_uncon_means.sim.h1 * 100
mean_first_period_female_uncon_means = np.mean(elas_marshall_female_uncon_means[:, 0])
elas_marshall_male_uncon_means = (model_uncon_means_10pct.sim.h2 - model_uncon_means.sim.h2) / model_uncon_means.sim.h2 * 100
mean_first_period_male_uncon_means = np.mean(elas_marshall_male_uncon_means[:, 0])

print(f"Marshall elasticity of hours worked Female in the first period (uncon_means): {mean_first_period_female_uncon_means}")
print(f"Marshall elasticity of hours worked Male in the first period (uncon_means): {mean_first_period_male_uncon_means}")

# Marshall elasticity for model_con_uni
elas_marshall_female_con_uni = (model_con_uni_10pct.sim.h1 - model_con_uni.sim.h1) / model_con_uni.sim.h1 * 100
mean_first_period_female_con_uni = np.mean(elas_marshall_female_con_uni[:, 0])
elas_marshall_male_con_uni = (model_con_uni_10pct.sim.h2 - model_con_uni.sim.h2) / model_con_uni.sim.h2 * 100
mean_first_period_male_con_uni = np.mean(elas_marshall_male_con_uni[:, 0])

print(f"Marshall elasticity of hours worked Female in the first period (con_uni): {mean_first_period_female_con_uni}")
print(f"Marshall elasticity of hours worked Male in the first period (con_uni): {mean_first_period_male_con_uni}")

# Marshall elasticity for model_con_means
elas_marshall_female_con_means = (model_con_means_10pct.sim.h1 - model_con_means.sim.h1) / model_con_means.sim.h1 * 100
mean_first_period_female_con_means = np.mean(elas_marshall_female_con_means[:, 0])
elas_marshall_male_con_means = (model_con_means_10pct.sim.h2 - model_con_means.sim.h2) / model_con_means.sim.h2 * 100
mean_first_period_male_con_means = np.mean(elas_marshall_male_con_means[:, 0])

print(f"Marshall elasticity of hours worked Female in the first period (con_means): {mean_first_period_female_con_means}")
print(f"Marshall elasticity of hours worked Male in the first period (con_means): {mean_first_period_male_con_means}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the models and increase relevant parameters by 10%
models = {
    "uncon_uni": model_uncon_uni,
    "con_uni": model_con_uni,
}

def increase_params_and_simulate(model, params):
    model_10pct = model.copy()
    for param in params:
        setattr(model_10pct.par, param, 1.10 * getattr(model.par, param))
    model_10pct.solve()
    model_10pct.simulate()
    return model_10pct

# Models with increased parameters
model_uncon_uni_10pct = increase_params_and_simulate(model_uncon_uni, ['uncon_uni'])
model_con_uni_10pct = increase_params_and_simulate(model_con_uni, ['cond'])

# Calculate elasticity for each model across all 10 periods
def calculate_elasticity(model_10pct, model):
    elas_marshall_female = (model_10pct.sim.h1 - model.sim.h1) / model.sim.h1 * 100
    elas_marshall_male = (model_10pct.sim.h2 - model.sim.h2) / model.sim.h2 * 100
    mean_elas_marshall_female = np.mean(elas_marshall_female, axis=0)
    mean_elas_marshall_male = np.mean(elas_marshall_male, axis=0)
    return mean_elas_marshall_female, mean_elas_marshall_male

mean_elas_female_uncon_uni, mean_elas_male_uncon_uni = calculate_elasticity(model_uncon_uni_10pct, model_uncon_uni)
mean_elas_female_con_uni, mean_elas_male_con_uni = calculate_elasticity(model_con_uni_10pct, model_con_uni)

# Plot the average elasticity across all 10 periods for each model
periods = np.arange(1, 11)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(periods, mean_elas_female_uncon_uni, label='Female', marker='o')
plt.plot(periods, mean_elas_male_uncon_uni, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
plt.title('Unconditional Transfers')
plt.legend()
plt.grid(True)
plt.xticks(periods)

plt.subplot(1, 2, 2)
plt.plot(periods, mean_elas_female_con_uni, label='Female', marker='o')
plt.plot(periods, mean_elas_male_con_uni, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
plt.title('Condtitional Transfers')
plt.legend()
plt.grid(True)
plt.xticks(periods)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the models and increase relevant parameters by 10%
models = {
    "uncon_uni": model_uncon_uni,
    "con_uni": model_con_uni,
}

def increase_params_and_simulate(model, params):
    model_10pct = model.copy()
    for param in params:
        setattr(model_10pct.par, param, 1.10 * getattr(model.par, param))
    model_10pct.solve()
    model_10pct.simulate()
    return model_10pct

# Models with increased parameters
model_uncon_uni_10pct = increase_params_and_simulate(model_uncon_uni, ['uncon_uni'])
model_con_uni_10pct = increase_params_and_simulate(model_con_uni, ['cond'])

# Calculate elasticity for each model across all 10 periods
def calculate_elasticity(model_10pct, model):
    elas_marshall_female = (model_10pct.sim.h1 - model.sim.h1) / model.sim.h1 * 100
    elas_marshall_male = (model_10pct.sim.h2 - model.sim.h2) / model.sim.h2 * 100
    mean_elas_marshall_female = np.mean(elas_marshall_female, axis=0)
    mean_elas_marshall_male = np.mean(elas_marshall_male, axis=0)
    return mean_elas_marshall_female, mean_elas_marshall_male

mean_elas_female_uncon_uni, mean_elas_male_uncon_uni = calculate_elasticity(model_uncon_uni_10pct, model_uncon_uni)
mean_elas_female_con_uni, mean_elas_male_con_uni = calculate_elasticity(model_con_uni_10pct, model_con_uni)

# Plot the average elasticity across all 10 periods for `uncon_uni`
periods = np.arange(1, 11)

plt.figure(figsize=(7, 6))
plt.plot(periods, mean_elas_female_uncon_uni, label='Female', marker='o')
plt.plot(periods, mean_elas_male_uncon_uni, label='Male', marker='o')


plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
#plt.title('uncon_uni')
plt.legend()
plt.grid(False)
plt.xticks(periods)
plt.tight_layout()
plt.show()

# Plot the average elasticity across all 10 periods for `con_uni`
plt.figure(figsize=(7, 6))
plt.plot(periods, mean_elas_female_con_uni, label='Female', marker='o')
plt.plot(periods, mean_elas_male_con_uni, label='Male', marker='o')
plt.xlabel('Period')
plt.ylabel('Average Marshall Elasticity (%)')
#plt.title('Conditional on market work')
plt.legend()
plt.grid(False)
plt.xticks(periods)
plt.tight_layout()
plt.show()
